In [1]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import Map, DropFields, ApplyMapping

sc = SparkContext()
context = GlueContext(sc)

In [2]:
path = "s3://network.cubo.datalake/airtable/raw/prod/"
path = path + "2023-03-21-00-08-11/data/"
path = path + "[Cubo Itaú] Conexões 💫/Time Cubo/Conexões/"
print(path)
dataframeConexoesFisicas = context.create_dynamic_frame.from_options(
                connection_type='s3',
                connection_options={
                    'paths': [path],
                    'recurse': True
                },
                format='json'
            )
dataframeConexoesFisicas.count()

s3://network.cubo.datalake/airtable/raw/prod/2023-03-21-00-08-11/data/[Cubo Itaú] Conexões 💫/Time Cubo/Conexões/


601

In [3]:
dataframeConexoesFisicas.printSchema()

root
|-- id: string
|-- createdTime: string
|-- fields: struct
|    |-- Canal Conexão: string
|    |-- Nome Parte 1: string
|    |-- category: string
|    |-- Email Parte 2: string
|    |-- Tipo Conector + Comunidade Conector Parte 2: string
|    |-- Responsável: string
|    |-- Companies 2: string
|    |-- Startup Cubo Parte 1: array
|    |    |-- element: string
|    |-- Companies: string
|    |-- Nome Parte 2: string
|    |-- Email Parte 1: string
|    |-- Startup Cubo Parte 2: array
|    |    |-- element: string
|    |-- Tipo Conector + Comunidade Conector Parte 1: string
|    |-- Contexto conexão: string
|    |-- Objetivo Conexão: string
|    |-- type: string
|    |-- Data da Conexão: string
|    |-- createdat: string
|    |-- action: string
|    |-- channel: string
|    |-- Corporate Cubo Parte 1: array
|    |    |-- element: string
|    |-- Parceiro Cubo Parte 1: array
|    |    |-- element: string
|    |-- Empresa Parte 2: string
|    |-- Empresa Parte 1: string
|    |-- Corpor

In [4]:
mapping = []
for item in dataframeConexoesFisicas.unnest().toDF().dtypes:
    if item[0].split('.')[0] == "fields":
        if len(item[0].split(".")) == 2:
            mapping.append((item[0], item[0].split('.')[1]))
    else:
        mapping.append((item[0], item[0].split('.')[0]))

In [6]:
newDf = dataframeConexoesFisicas.apply_mapping(mapping).resolveChoice([("createdat", "cast:string")]) \
        .drop_fields(["Empresa Parte 1", "Startup Cubo Parte 1", "Corporate Cubo Parte 1", "Parceiro Cubo Parte 1",
        "Empresa Parte 2", "Startup Cubo Parte 2", "Corporate Cubo Parte 2", "Parceiro Cubo Parte 2"]) \
        .rename_field("Companies", "slug")

In [7]:
newDf.printSchema()

root
|-- id: string
|-- createdTime: string
|-- Canal Conexão: string
|-- Nome Parte 1: string
|-- category: string
|-- Email Parte 2: string
|-- Tipo Conector + Comunidade Conector Parte 2: string
|-- Responsável: string
|-- Companies 2: string
|-- Nome Parte 2: string
|-- Email Parte 1: string
|-- Tipo Conector + Comunidade Conector Parte 1: string
|-- Contexto conexão: string
|-- Objetivo Conexão: string
|-- type: string
|-- Data da Conexão: string
|-- action: string
|-- channel: string
|-- createdat: string
|-- slug: string

